In [ ]:
import os
import pandas as pd
import numpy as np
from root_numpy import root2array 
import uproot
from tqdm import tqdm
import matplotlib.pyplot as plt
from pyjet import cluster
from pyjet.testdata import get_event

In [ ]:
def jet_trimmer_2J(event, R0, R1, pt_cut):
    # R0 = Clustering radius for the main jets
    # R1 = Clustering radius for the subjets in the primary jet
    # pt_cut = Threshold for subjets (relative to the primary jet it's a subjet of)    
    flattened_event = flatten(event)
    sequence = cluster(flattened_event, R=R0, p= -1)
    # Main jets
    jets = sequence.inclusive_jets()
    # In case we are missing a leading jet, break early
    if len(jets) < 2:
        return []
    

    # Take just the leading jet

    j0 = jet_trim(jets[0])
    j1 = jet_trim(jets[1])
    
    if (len(j0)==0)or(len(j1)==0):
        return []
    else:
        return [j0[0],j1[0]]

def flatten(event): #my update
    fp = np.concatenate((np.expand_dims(event[0], axis=-1),
                         np.expand_dims(event[1], axis=-1),
                         np.expand_dims(event[2], axis=-1),
                         np.expand_dims(event[3], axis=-1)), axis=-1)
    fp = fp.transpose((1,0))
    fp = np.core.records.fromarrays( [fp[:][0],fp[:][1],fp[:][2],fp[:][3]], names= 'pT, eta, phi, mass' , formats = 'f8, f8, f8,f8')

    return fp

def jet_clust(event, R0, p=-1):
    # R0 = Clustering radius for the main jets
    # R1 = Clustering radius for the subjets in the primary jet
    # pt_cut = Threshold for subjets (relative to the primary jet it's a subjet of)    
    flattened_event = flatten(event)
    sequence = cluster(flattened_event, R=R0, p= p)
    # Main jets
    jets = sequence.inclusive_jets()
    # In case we are missing a leading jet, break early
    if len(jets) < 2:
        return []
    

    # Take just the leading jet
    jet0 = jets[0]
    jet1 = jets[1]

    return [jet0, jet1]

def jet_trim(jet0):
    # Define a cut threshold that the subjets have to meet (i.e. 5% of the original jet pT)
    jet0_max = jet0.pt
    jet0_cut = jet0_max*pt_cut

    # Grab the subjets by clustering with R1
    subjets = cluster(jet0.constituents_array(), R=R1, p=1)
    subjet_array = subjets.inclusive_jets()
    j0 = []
    if (subjet_array[0].pt >= jet0_cut):
#         j0 = subjet_array[0].constituents_array()
        for ij, subjet in enumerate(subjet_array):
#             if (ij == 0):
#                 continue
            if subjet.pt < jet0_cut:
                # subjet doesn't meet the percentage cut on the original jet pT
                continue
            if subjet.pt >= jet0_cut:
                # Get the subjets pt, eta, phi constituents
                subjet_data = subjet.constituents_array()
                j0.append(subjet_data)
#                 j0 = np.append(j0, subjet_data)
    else:
        j0 = subjet_array[0].constituents_array()*0
    jet = j0[0]
    for i, subjet in enumerate(j0):
        if i==0 :
            continue
        jet = np.append(jet, subjet)
        
    sequence = cluster(jet, R=R0, p=-1)
    jet = sequence.inclusive_jets()
    return jet

def jet_clust_all(event, R0, p=-1):
    # R0 = Clustering radius for the main jets
    # R1 = Clustering radius for the subjets in the primary jet
    # pt_cut = Threshold for subjets (relative to the primary jet it's a subjet of)    
    flattened_event = flatten(event)
    sequence = cluster(flattened_event, R=R0, p= -1)
    # Main jets
    jets = sequence.inclusive_jets()
    # In case we are missing a leading jet, break early

    return jets

In [ ]:
def jet_selection(jet, W):
    m_inv = []
    m_T = []
    Wc = []
    for i in tqdm(range(len(jet))):
        if jet[i][0].shape[0]<2: #at least two jet
            continue
        if jet[i][_JPT][0]<440: # leading jet pt >= 440 Gev
            continue
        if jet[i][_JPT][1]<60: #sub-leading jet pt >= 60 Gev
            continue
        if abs(jet[i][_JEta][0]-jet[i][_JEta][1])>=1.2:
            continue
        m_inv.append(mass_inv(jet[i]))
        m_T.append(mass_T(jet[i]))
        Wc.append(W[i])
    return m_inv, m_T, Wc

def jet_selection2(jet, W):
    m_inv = []
#     m_T = []
    Wc = []
    for i in tqdm(range(len(jet))):
        if len(jet[i])<2: #at least two jet
            continue
        if jet[i][0].pt<440: # leading jet pt >= 440 Gev
            continue
        if jet[i][1].pt<60: #sub-leading jet pt >= 60 Gev
            continue
        if abs(jet[i][0].eta-jet[i][1].eta)>=1.2:
            continue
        m_inv.append(mass_inv_s(jet[i][0],jet[i][1]))
#         m_T.append(mass_T_s(jet[i]))
        Wc.append(W[i])
    return m_inv, Wc

In [ ]:
def mass_inv_s(j0, j1):
    
    M_inv = (j0.e+j1.e)**2 -(j0.px+j1.px)**2 - (j0.py+j1.py)**2 - (j0.pz+j1.pz)**2


    return M_inv**0.5

def mass_inv(jet):
#    ["FatJet.PT", "FatJet.Eta", "FatJet.Phi", "FatJet.Mass"] 
# formula https://en.wikipedia.org/wiki/Invariant_mass#As_defined_in_particle_physics
#     M_inv = 2*jet[_JPT][0]*jet[_JPT][1]*(np.cosh(jet[_JEta][0]-jet[_JEta][1])-np.cos(jet[_JPhi][0]-jet[_JPhi][1]))
    M_inv = jet[_JMass][0]**2+jet[_JMass][1]**2+ 2*(ET_np(jet[_JPT][0], jet[_JMass][0])*
                                                    ET_np(jet[_JPT][1], jet[_JMass][1])* np.cosh(jet[_JEta][0]-jet[_JEta][1]) 
                                                    - jet[_JPT][0]*jet[_JPT][1] ) 

    e, px, py, pz = transform_np(jet[_JPT][0], jet[_JEta][0], jet[_JPhi][0], jet[_JMass][0])
    e2, px2, py2, pz2 = transform_np(jet[_JPT][1], jet[_JEta][1], jet[_JPhi][1], jet[_JMass][1])
    M_inv = (e+e2)**2 -(px+px2)**2 - (py+py2)**2 - (pz+pz2)**2


    return M_inv**0.5
# def inv_mass(pt1,pt2,eta1,eta2,phi1,phi2,m1,m2):
#     m = m1**2+m2**2+ 2*(ET_np(pt1, m1)*ET_np(pt2, m2)* np.cosh(eta1-eta2) - pt1*pt2 ) 
#     return m**0.5
def ET_np(pt,m):
    return(pt**2+m**2)**0.5

def mas_in(jet,n):
#    ["FatJet.PT", "FatJet.Eta", "FatJet.Phi", "FatJet.Mass"] 
# formula https://en.wikipedia.org/wiki/Invariant_mass#As_defined_in_particle_physics
    M_inv = 2*jet[_JPT][n]*jet[_JPT][n]*(np.cosh(jet[_JEta][n]-jet[_JEta][n])-np.cos(jet[_JPhi][n]-jet[_JPhi][n]))
    return M_inv**0.5
def ET(jet,n):
    return (mas_in(jet,n)**2+jet[_JPT][n]**2)**0.5
def deltaPhi(phi1,phi2):
    x = phi1-phi2
    while x>= np.pi: x -= np.pi*2.
    while x< -np.pi: x += np.pi*2.
    return x    

def mass_T(jet):
    M_T=2*ET(jet,1)*ET(jet,0)*(1-np.cos(deltaPhi(jet[_JPhi][1],jet[_JPhi][0])) ) 
    return  M_T**0.5

def transform_np(pt, eta, phi, mass):
    px= pt*np.cos(phi)
    py= pt*np.sin(phi)
    pz = pt*np.sinh(eta)
    p = pt*np.cosh(eta)
    e = (p**2+mass**2)**0.5
    return e,px,py,pz


def mass_inv_all(jets):
    E, PX, PY, PZ = 0, 0, 0, 0
    for i in jets:
        E = E + i.e
        PX = PX + i.px
        PY = PY + i.py
        PZ = PZ + i.pz
    
    M_inv = E**2 -PX**2 - PY**2 - PZ**2


    return M_inv**0.5


In [ ]:
def jet_trimmer_test(event, R0, R1, pt_cut):
    # R0 = Clustering radius for the main jets
    # R1 = Clustering radius for the subjets in the primary jet
    # pt_cut = Threshold for subjets (relative to the primary jet it's a subjet of)    
    flattened_event = flatten(event)
    sequence = cluster(flattened_event, R=R0, p= -1)
    # Main jets
    jets = sequence.inclusive_jets()
    # In case we are missing a leading jet, break early
    if len(jets) < 2:
        return []
    

    # Take just the leading jet

    j0 = jet_trim(jets[0])
    j1 = jet_trim(jets[1])
    
    return j0

In [ ]:
N=20
pt_cut= 0.03
R0 = 0.5
R1 = 0.2
_JPT, _JEta, _JPhi, _JMass = 0, 1, 2, 3
m_inv, m_T, WTc, m_inv2, m_T2, WTc2 = [], [], [], [], [], []
jetT, WT, jetT2, WT2 = [], [], [], []

for i in tqdm(range(N)):
    root_file = "/Storage/james/ROOT/Reproduce/SVJ/rinv03/Z1500/test"+str(i)+".root"
    file = uproot.open(root_file)
    root_file2 = "/Storage/james/ROOT/Reproduce/SVJ/rinv0/Z1500/test"+str(i)+".root"
    
    file2 = uproot.open(root_file2)
    W = file["Delphes;1"]["Event.Weight"].array()

    events = np.array([np.array(file["Delphes;1"]["Tower.ET"].array()),
                       np.array(file["Delphes;1"]["Tower.Eta"].array()),
                       np.array(file["Delphes;1"]["Tower.Phi"].array()),
                       np.array(file["Delphes;1"]["Tower.E"].array())*0  #assume m<<1
                      ])
    
    
    events = np.expand_dims(events, axis=-1)
    events = events.transpose((1,0,2))
    events = np.squeeze(events,axis=(2,))
    WT = np.append(WT,W, axis=0)
    

        
    
    for j in range(len(events)):
        jetL = jet_trimmer_2J(events[j], R0, R1, pt_cut)
        jetT.append(jetL)
